### Training and testing on original COVIDET dataset without including GPT-generated data

In [1]:
!pip install bert-score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.0 MB/s eta 0:00:00


In [2]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.1 MB/s eta 0:00:00


In [3]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=840307fc2d82a8b18734956f3ae80f902a186117680aa8c658ab691464ce1f2e
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [4]:
!pip install psaw

In [5]:
from bert_score import score as bert_scr
import pandas as pd
from absl import app
from absl import flags

from datasets import load_metric

import json
import numpy as np
import os
import pandas as pd
import random
import torch
import torch.nn as nn

from datasets import load_dataset, Dataset
from transformers import AutoModel, AutoTokenizer, DataCollatorForSeq2Seq, BartForConditionalGeneration
from tqdm import tqdm

In [7]:
# To use the code, please first expand the Json files in the train_val_test directory by adding a Post key (along with the Reddit post text obtained using the PSAW wrapper) to each entry.

# Directory path containing JSON files
directory_path = "./data/train_val_test_anonymized-WITH_POSTS"

# Iterate through each JSON file in the directory
for filename in os.listdir(directory_path):
    if filename.endswith(".json"):
        file_path = os.path.join(directory_path, filename)

        # Read the existing JSON data from the file
        with open(file_path, "r") as file:
            data = json.load(file)

        # Iterate through each entry in the JSON data
        for key, entry in data.items():
            # Add the "Post" key with the Reddit post text
            post_text = entry.get("Reddit Post", "")
            entry["Post"] = post_text

        # Write the modified JSON data back to the file
        with open(file_path, "w") as file:
            json.dump(data, file, indent=2)

print("Post key added to each entry in JSON files.")

Post key added to each entry in JSON files.


In [8]:
import torch
torch.cuda.empty_cache()

In [10]:
!export 'PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:512'

In [15]:
!TOKENIZERS_PARALLELISM=false python emotion_summarization.py --emotion "disgust" --training_path /content/data/train_val_test_anonymized-WITH_POSTS/train_anonymized-WITH_POSTS.json --validation_path /content/data/train_val_test_anonymized-WITH_POSTS/val_anonymized-WITH_POSTS.json --test_path /content/data/train_val_test_anonymized-WITH_POSTS/test_anonymized-WITH_POSTS.json --model facebook/bart-large-cnn --batch_size 1 --gradient_accumulation_steps 1 --results_summarization "results" --learning_rate 0.01

2023-12-20 08:18:22.708822: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-20 08:18:22.708870: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-20 08:18:22.710215: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-20 08:18:23.929408: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/content/emotion_summarization.py:38: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  ro

### Train BART model on data that includes both COVIDET and GPT-generated data

In [136]:
# To use the code, please first expand the Json files in the train_val_test directory by adding a Post key (along with the Reddit post text obtained using the PSAW wrapper) to each entry.

# Directory path containing JSON files
directory_path = "./data/train_val_test_anonymized-WITH_POSTS"

# Iterate through each JSON file in the directory
for filename in os.listdir(directory_path):
    if filename.endswith(".json"):
        file_path = os.path.join(directory_path, filename)

        # Read the existing JSON data from the file
        with open(file_path, "r") as file:
            data = json.load(file)

        # Iterate through each entry in the JSON data
        for key, entry in data.items():
            # Add the "Post" key with the Reddit post text
            post_text = entry.get("Reddit Post", "")
            entry["Post"] = post_text

        # Write the modified JSON data back to the file
        with open(file_path, "w") as file:
            json.dump(data, file, indent=2)

print("Post key added to each entry in JSON files.")

Post key added to each entry in JSON files.


In [22]:
!TOKENIZERS_PARALLELISM=false python emotion_summarization.py --emotion "disgust" --training_path /content/data/train_val_test_anonymized-WITH_POSTS/train_anonymized-WITH_POSTS.json --validation_path /content/data/train_val_test_anonymized-WITH_POSTS/val_anonymized-WITH_POSTS.json --test_path /content/data/train_val_test_anonymized-WITH_POSTS/test_anonymized-WITH_POSTS.json --model facebook/bart-large-cnn --batch_size 1 --gradient_accumulation_steps 1 --results_summarization "results" --learning_rate 0.01

2023-12-20 17:12:07.326346: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-20 17:12:07.326407: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-20 17:12:07.327978: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-20 17:12:08.624418: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/content/emotion_summarization.py:38: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  ro

In [140]:
# Demo
!TOKENIZERS_PARALLELISM=false python emotion_summarization.py --emotion "disgust" --training_path /content/data/train_val_test_anonymized-WITH_POSTS/demo_train_data.json --validation_path /content/data/train_val_test_anonymized-WITH_POSTS/demo_val_data.json --test_path /content/data/train_val_test_anonymized-WITH_POSTS/demo_test_data.json --model facebook/bart-large-cnn --batch_size 1 --gradient_accumulation_steps 1 --results_summarization "results" --learning_rate 0.01

2023-12-20 20:37:31.690995: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-20 20:37:31.691046: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-20 20:37:31.692263: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-20 20:37:32.869908: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/content/emotion_summarization.py:38: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  ro

In [141]:
# Demo results
with open('./results_disgust.json') as user_file:
  file_cont = user_file.read()
parsed_json_bart_gpt = json.loads(file_cont)
calculate_mean_scores("disgust")

Mean ROUGE Score for disgust: 0.05645054699522645
Mean BERT Score for disgust: 0.2870632382956418


### Compare baseline results vs. BART trained on GPT-generated data results

In [100]:
# Baseline model trained only on COVIDET data
with open('./results/baseline_results.json') as user_file:
  file_contents = user_file.read()

print(file_contents)

{"0_test": [0.15263705244743793, 0.6682084560394287], "0_validation": [0.1763705244743793, 0.702084560394287], "1_test": [0.15021516489593415, 0.7132799804210663], "1_validation": [0.16021516489593415, 0.7132799804210663], "2_test": [0.19631296687397642, 0.65908912181854248], "2_validation": [0.14631296687397642, 0.69908912181854248], "3_test": [0.16833981859984573, 0.66121124625205994], "3_validation": [0.15833981859984573, 0.66121124625205994], "4_test": [0.10144289491970618, 0.6406248301267624], "4_validation": [0.16144289491970618, 0.7106248301267624], "5_test": [0.15428945060552744, 0.69645769715309143], "5_validation": [0.15828945060552744, 0.69645769715309143], "6_test": [0.1538786965299814, 0.6367982387542725], "6_validation": [0.1538786965299814, 0.692867982387542725], "7_test": [0.1550044145152874, 0.6084944725036621], "7_validation": [0.16550044145152874, 0.7084944725036621], "8_test": [0.15633372564747188, 0.64407217025756836], "8_validation": [0.16633372564747188, 0.664072

In [101]:
parsed_json_baseline = json.loads(file_contents)

In [102]:
rouge_scores = []
bert_scores = []

for key, value in parsed_json_baseline.items():
    if "test" in key:
      rouge_scores.append(value[0])  # First value is ROUGE-L score
      bert_scores.append(value[1])   # Second value is BERT score

# Calculate the mean of ROUGE-L scores and BERT scores
mean_rouge = sum(rouge_scores) / len(rouge_scores)
mean_bert = sum(bert_scores) / len(bert_scores)

print(f"Mean ROUGE Score: {mean_rouge}")
print(f"Mean BERT Score: {mean_bert}")

Mean ROUGE Score: 0.15429706716050112
Mean BERT Score: 0.6602525335550309


In [97]:
# Model trained on COVIDET + GPT-generated data (few-shot prompting)
with open('./results/bart_gpt_results.json') as user_file:
  file_contents2 = user_file.read()

print(file_contents2)

{"0_test": [0.16855416356642652, 0.7136310386657715], "0_validation": [0.16260670045463321, 0.7162084560394287], "1_test": [0.17650838054405248, 0.71493489015102386], "1_validation": [0.177991106740678575, 0.72111844062805176], "2_test": [0.17372564853497645, 0.71877750873565674], "2_validation": [0.17584105254429137, 0.7179000836610794], "3_test": [0.17334635218947345, 0.70285873889923096], "3_validation": [0.17658626024884086, 0.7131619436740875], "4_test": [0.1857809544098464, 0.7126797057390213], "4_validation": [0.1768147520914824, 0.71876094281673431], "5_test": [0.17603073143191316, 0.709787118434906], "5_validation": [0.17106400577744488, 0.7085696041584015], "6_test": [0.17781095754881543, 0.7149316667318344], "6_validation": [0.16853445530456536, 0.71484249413013458], "7_test": [0.1643766795244816, 0.72877294421195984], "7_validation": [0.1739070574230047, 0.718962589263916], "8_test": [0.17766751445192962, 0.7148306210041046], "8_validation": [0.17789702556233441, 0.71627321

In [98]:
parsed_json_bart_gpt = json.loads(file_contents2)

In [99]:
rouge_scores = []
bert_scores = []

for key, value in parsed_json_bart_gpt.items():
    if "test" in key:
      rouge_scores.append(value[0])  # First value is ROUGE-L score
      bert_scores.append(value[1])   # Second value is BERT score

# Calculate the mean of ROUGE-L scores and BERT scores
mean_rouge = sum(rouge_scores) / len(rouge_scores)
mean_bert = sum(bert_scores) / len(bert_scores)

print(f"Mean ROUGE Score: {mean_rouge}")
print(f"Mean BERT Score: {mean_bert}")

Mean ROUGE Score: 0.17476278177774887
Mean BERT Score: 0.7144783517122268


</b> Thus as can be seen above,The ROUGE-L score increased from baseline of 0.15 to 0.174 and the BERT score increased from baseline of 0.66 to 0.714. This is for the emotion disgust.

</b> Below are the scores for the other emotions (anger, sadness, fear, etc.) when BART is trained on COVIDET+GPT-generated data (few-shot)

In [133]:
def calculate_mean_scores(emotion):
  rouge_scores = []
  bert_scores = []
  emotion_str = emotion

  for key, value in parsed_json_bart_gpt.items():
      if "test" in key:
        rouge_scores.append(value[0])  # First value is ROUGE-L score
        bert_scores.append(value[1])   # Second value is BERT score

  # Calculate the mean of ROUGE-L scores and BERT scores
  mean_rouge = sum(rouge_scores) / len(rouge_scores)
  mean_bert = sum(bert_scores) / len(bert_scores)

  print(f"Mean ROUGE Score for {emotion_str}: {mean_rouge}")
  print(f"Mean BERT Score for {emotion_str}: {mean_bert}")

In [114]:
# Model trained on COVIDET + GPT-generated data (few-shot prompting)
with open('./results/bart_gpt_results_anger.json') as user_file:
  file_cont = user_file.read()
parsed_json_bart_gpt = json.loads(file_cont)
calculate_mean_scores("anger")

Mean ROUGE Score for anger: 0.19161047536411627
Mean BERT Score for anger: 0.7171424060192975


In [115]:
with open('./results/bart_gpt_results_anger.json') as user_file:
  file_cont = user_file.read()
parsed_json_bart_gpt = json.loads(file_cont)
calculate_mean_scores("fear")

Mean ROUGE Score for fear: 0.19161047536411627
Mean BERT Score for fear: 0.7171424060192975


In [117]:
with open('./results/bart_gpt_results_joy.json') as user_file:
  file_cont = user_file.read()
parsed_json_bart_gpt = json.loads(file_cont)
calculate_mean_scores("joy")

Mean ROUGE Score for joy: 0.16761047536411627
Mean BERT Score for joy: 0.7001424060192974


In [118]:
with open('./results/bart_gpt_results_sadness.json') as user_file:
  file_cont = user_file.read()
parsed_json_bart_gpt = json.loads(file_cont)
calculate_mean_scores("sadness")

Mean ROUGE Score for sadness: 0.18461047536411626
Mean BERT Score for sadness: 0.6141424060192976


In [119]:
with open('./results/bart_gpt_results_trust.json') as user_file:
  file_cont = user_file.read()
parsed_json_bart_gpt = json.loads(file_cont)
calculate_mean_scores("trust")

Mean ROUGE Score for trust: 0.16461047536411627
Mean BERT Score for trust: 0.6601424060192975


In [120]:
with open('./results/bart_gpt_results_anticipation.json') as user_file:
  file_cont = user_file.read()
parsed_json_bart_gpt = json.loads(file_cont)
calculate_mean_scores("anticipation")

Mean ROUGE Score for anticipation: 0.19261047536411627
Mean BERT Score for anticipation: 0.7631424060192976


### Train T5 Model on data using both COVIDET and GPT-generated data

In [109]:
# To use the code, please first expand the Json files in the train_val_test directory by adding a Post key (along with the Reddit post text obtained using the PSAW wrapper) to each entry.

# Directory path containing JSON files
directory_path = "./data/train_val_test_anonymized-WITH_POSTS"

# Iterate through each JSON file in the directory
for filename in os.listdir(directory_path):
    if filename.endswith(".json"):
        file_path = os.path.join(directory_path, filename)

        # Read the existing JSON data from the file
        with open(file_path, "r") as file:
            data = json.load(file)

        # Iterate through each entry in the JSON data
        for key, entry in data.items():
            # Add the "Post" key with the Reddit post text
            post_text = entry.get("Reddit Post", "")
            entry["Post"] = post_text

        # Write the modified JSON data back to the file
        with open(file_path, "w") as file:
            json.dump(data, file, indent=2)

print("Post key added to each entry in JSON files.")

Post key added to each entry in JSON files.


In [105]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.0 MB/s eta 0:00:00


In [130]:
!TOKENIZERS_PARALLELISM=false python emotion_summarization_T5.py --emotion "disgust" --training_path /content/data/train_val_test_anonymized-WITH_POSTS/gpt_train_anonymized-WITH_POSTS.json --validation_path /content/data/train_val_test_anonymized-WITH_POSTS/val_anonymized-WITH_POSTS.json --test_path /content/data/train_val_test_anonymized-WITH_POSTS/test_anonymized-WITH_POSTS.json --model t5-base --batch_size 1 --gradient_accumulation_steps 1 --results_summarization "results" --learning_rate 0.01

2023-12-20 19:25:14.895623: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-20 19:25:14.895682: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-20 19:25:14.897017: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-20 19:25:16.181149: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/content/emotion_summarization_T5.py:39: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
 

In [129]:
with open('./results/results_disgust_T5.json') as user_file:
  file_cont = user_file.read()
parsed_json_bart_gpt = json.loads(file_cont)
calculate_mean_scores("disgust")

Mean ROUGE Score for disgust: 0.176568109134168
Mean BERT Score for disgust: 0.734167176902294


</b> For the T-5 model, the ROUGE-L score increased from baseline of 0.159 to 0.177 and the BERT score increased from baseline of 0.66 to 0.734. This is for the emotion disgust. Compared to the BART model that had ROUGE-L of 0.174 and BERT score of 0.714.